<a href="https://colab.research.google.com/github/sulaimangm/SEP769_CPS_DeepLearning/blob/main/SEP769_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q kaggle

In [2]:
!mkdir ~/.kaggle

In [6]:
!cp /content/kaggle.json ~/.kaggle/kaggle.json

In [7]:
!chmod 600 ~/.kaggle/kaggle.json

In [8]:
!ls ~/.kaggle

kaggle.json


In [10]:
# Download the wildfire prediction dataset
!kaggle datasets download -d garystafford/environmental-sensor-data-132k

# Unzip the downloaded dataset
!unzip environmental-sensor-data-132k.zip

environmental-sensor-data-132k.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  environmental-sensor-data-132k.zip
  inflating: iot_telemetry_data.csv  


In [11]:
import zipfile

# Set the path to the zip file
zip_file_path = '/content/environmental-sensor-data-132k.zip'

# Set the path to the directory where the contents of the zip file will be extracted
extract_path = '/content/environmental-sensor-data-132k'

# Extract the contents of the zip file to the extract directory
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Import Libraries

In [12]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [14]:
dataset_path = '/content/iot_telemetry_data.csv'
df = pd.read_csv(dataset_path)
df.head()

,ts,device,co,humidity,light,lpg,motion,smoke,temp
0,1.594512e+09,b8:27:eb:bf:9d:51,0.004956,51.000000,False,0.007651,False,0.020411,22.700000
1,1.594512e+09,00:0f:00:70:91:0a,0.002840,76.000000,False,0.005114,False,0.013275,19.700001
2,1.594512e+09,b8:27:eb:bf:9d:51,0.004976,50.900000,False,0.007673,False,0.020475,22.600000
3,1.594512e+09,1c:bf:ce:15:ec:4d,0.004403,76.800003,True,0.007023,False,0.018628,27.000000
4,1.594512e+09,b8:27:eb:bf:9d:51,0.004967,50.900000,False,0.007664,False,0.020448,22.600000


In [ ]:
encLab = LabelEncoder()
deviceEnc = pd.DataFrame(encLab.fit_transform(data['device']), columns=['condition'])
deviceEnc.index = df.index

In [15]:
classes = df['device']
df.drop(columns=['device'],inplace=True)
classes

0         b8:27:eb:bf:9d:51
1         00:0f:00:70:91:0a
2         b8:27:eb:bf:9d:51
3         1c:bf:ce:15:ec:4d
4         b8:27:eb:bf:9d:51
                ...        
405179    00:0f:00:70:91:0a
405180    b8:27:eb:bf:9d:51
405181    1c:bf:ce:15:ec:4d
405182    00:0f:00:70:91:0a
405183    b8:27:eb:bf:9d:51
Name: device, Length: 405184, dtype: object

In [ ]:
encLab = LabelEncoder()
deviceEnc = pd.DataFrame(encLab.fit_transform(df['device']), columns=['condition'])
deviceEnc.index = df.index